In [1]:
# LOAD Packages 
import pandas as pd
import numpy as np
import awkward as ak
!pip install awkward_pandas
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import os
import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import json
import h5py
import sys
from mpl_toolkits.axes_grid1 import make_axes_locatable
!pip install plotly
import plotly
import plotly.graph_objects as go

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Open edep-sim file: 
edep_sim_file = "/global/cfs/cdirs/dune/users/ehinkle/nd_prototypes_ana/sheep-model/sample_gen/test_electron.hdf5"
sim_h5 = h5py.File(edep_sim_file, 'r')

In [ ]:
class ShowerDataset(Dataset):
         
    def __init__(self, h5_file_dir, h5_file_name):

        self.file_dir = h5_file_dir
        self.h5_file_name = h5_file_name
        self.h5_file_path = os.path.join(self.file_dir, self.h5_file_name)
        self.h5_file = h5py.File(self.h5_file_path, 'r')
        self.events = self.h5_file['events']
        self.trajectories = self.h5_file['trajectories']
        self.segments = self.h5_file['segments']
        

    def __len__(self):
        return len(self.events)

    def __getitem__(self, idx):
        event = self.events[idx]
        event_id = event['event_id']
        trajectories = self.trajectories[self.trajectories['event_id'] == event_id]
        segments = self.segments[self.segments['event_id'] == event_id]
        true_KE_initial = np.sqrt(np.sum(np.square(event['pxyz_start'])))
        seg_dE = segments['dE']
        seg_x_start = segments['x_start']
        seg_y_start = segments['y_start']
        seg_z_start = segments['z_start']
        seg_x_end = segments['x_end']
        seg_y_end = segments['y_end']
        seg_z_end = segments['z_end']
        event_data = {
            'event_id': event_id,
            'pxyz_start': event['pxyz_start'],
            'pxyz_end': event['pxyz_end'],
            'true_KE_initial': true_KE_initial,
            'segments': {
                'dE': seg_dE,
                'x_start': seg_x_start,
                'y_start': seg_y_start,
                'z_start': seg_z_start,
                'x_end': seg_x_end,
                'y_end': seg_y_end,
                'z_end': seg_z_end
            }
        }
        return event_data